In [ ]:
import pandas as pd
import numpy as np

dat = pd.read_csv('Data/subject_data.csv')
full_curves = np.load('Data/full-curves.npy')
baseline = dat[['IntraCranialVol', 'lhCortexVol', 'rhCortexVol', 'CortexVol',
       'SubCortGrayVol', 'TotalGrayVol', 'SupraTentorialVol',
       'lhCorticalWhiteMatterVol', 'rhCorticalWhiteMatterVol',
       'CorticalWhiteMatterVol']]

## Residuals

In [ ]:
from sklearn.model_selection import train_test_split
from IPython.display import display, Math


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import balanced_accuracy_score, f1_score
bacc = []
f1 = []
# No. samples to use to train regression model
n = 200
# Select features to use to train model GM: 0:300, WM: 300:600, Point cloud: 600:800
features = full_curves[:, :600]
# features = baseline
classifier = SVC(kernel='rbf', probability=True, class_weight='balanced')
regressor = LinearRegression()

for _ in range(5):
    Xtrain, Xtest, ytrain, ytest = train_test_split(features,dat,test_size=0.2)
    
    # Isolate data for regression model
    ytrain = ytrain.reset_index(drop=True)
    healthy = ytrain[~ytrain['isSick']].sample(n, replace=False)
    mask = np.zeros((ytrain.shape[0]), dtype=bool)
    mask[healthy.index] = 1
    Xtrain_reg = Xtrain[mask]

    regressor.fit(healthy['Age'].to_numpy().reshape(-1, 1), Xtrain_reg)

    # Fit classification model, predict results and compute scores
    ytrain_class = ytrain[~ytrain.index.isin(healthy.index)]                
    classifier.fit(Xtrain[~mask] - regressor.predict(ytrain_class['Age'].to_numpy().reshape(-1, 1)), ytrain_class['isSick'])
    predictions = classifier.predict(Xtest - regressor.predict(ytest['Age'].to_numpy().reshape(-1, 1)))
    bacc.append(balanced_accuracy_score(ytest['isSick'], predictions))
    f1.append(f1_score(ytest['isSick'], predictions))
    

display(Math(r'\text{{Balanced accuracy: }}{} \pm {}'.format(np.mean(bacc), np.std(bacc))))
display(Math(r'\text{{F1: }}{} \pm {}'.format(np.mean(f1), np.std(f1))))

## Raw

In [ ]:
# Compute model performance on raw Betti curves
bacc = []
f1 = []

for _ in range(5):
    Xtrain, Xtest, ytrain, ytest = train_test_split(full_curves[:, :400],dat['isSick'], test_size=0.2, stratify=dat['isSick'])

    classifier.fit(Xtrain, ytrain)
    predictions = classifier.predict(Xtest)
    bacc.append(balanced_accuracy_score(ytest, predictions))
    f1.append(f1_score(ytest, predictions))
display(Math(r'\text{{Balanced accuracy: }}{} \pm {}'.format(np.mean(bacc), np.std(bacc))))
display(Math(r'\text{{F1: }}{} \pm {}'.format(np.mean(f1), np.std(f1))))

## Age prediction

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import balanced_accuracy_score, f1_score, mean_absolute_error, r2_score

mae = []
r2 = []
# Use only healthy participants
ages = dat[~dat['isSick']]['Age']
# curves = baseline.iloc[ages.index]
curves = full_curves[ages.index, :300]
regressor = RandomForestRegressor()

for _ in range(5):
    Xtrain, Xtest, ytrain, ytest = train_test_split(curves,ages, test_size=0.2)

    regressor.fit(Xtrain, ytrain)
    predictions = regressor.predict(Xtest)
    m = mean_absolute_error(ytest, predictions)
    r = r2_score(ytest, predictions)
    mae.append(m)
    r2.append(r)
    print(m, r)
display(Math(r'\text{{MAE: }}{} \pm {}'.format(np.mean(mae), np.std(mae))))
display(Math(r'\text{{r2: }}{} \pm {}'.format(np.mean(r2), np.std(r2))))